__Created and Maintained by Boon < <boon.siew@illumio.com> >__

__NOTE:__
This script generates the following outputs:<br>
    1. CSV with consolidated flows output - "consolidated_output.csv"<br>
    2. CSV with consolidated flows output without ports and protocols - "consolidated_output-noports.csv"<br>
    3. CSV with Source VENs without APP label assigned - "Source_hostname_without_applabel_output.csv"<br>
    4. CSV with Destination VENs without APP label assigned - "Destination_hostname_without_applabel_output.csv"<br>
    5. HTML with graph - "sunburst_chart_with_legend.html"
    
- pip3 install -U kaleido
- pip3 install -U pandas
- pip3 install -U plotly

**Update the following Variables:**

In [ ]:
csv_file_path="Remote/172.21.1/outbound.csv"

Should we enable the location view? (default is False)

In [ ]:
location=True

Sunburst chart depth level (default 2, upto 6)

In [ ]:
maxdepth=6

<span style='background:Red;font-weight:bold;'> ### DO NOT EDIT FROM HERE ###</span>

In [ ]:
import sys
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
def read_csv_file(file_path):
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        sys.exit(1)

In [ ]:
def preprocess_data(df):
    # Condition 1: If "Source Hostname" and "Destination Hostname" are not empty, fill in with "NO LABEL" for certain columns if any one of them is empty.
    hostname_columns = ['Source Application', 'Source Environment', 'Source Location', 'Destination Application', 'Destination Environment', 'Destination Location']
    df.loc[~df['Source Hostname'].isna() & df['Source Hostname'].notna() &
           ~df['Destination Hostname'].isna() & df['Destination Hostname'].notna(),
           hostname_columns] = df[hostname_columns].fillna('NO LABEL')

    # Condition 2: If "Source IPList" is not empty, fill in with "NO LABEL" for certain columns if any one of them is empty.
    Source_iplist_columns = ['Destination Application', 'Destination Environment', 'Destination Location']
    df.loc[~df['Source IPList'].isna(), Source_iplist_columns] = df[Source_iplist_columns].fillna('NO LABEL')

    # Condition 3: If "Destination IPList" is not empty, fill in with "NO LABEL" for certain columns if any one of them is empty.
    Destination_iplist_columns = ['Source Application', 'Source Environment', 'Source Location']
    df.loc[~df['Destination IPList'].isna(), Destination_iplist_columns] = df[Destination_iplist_columns].fillna('NO LABEL')

    df['First Detected'] = pd.to_datetime(df['First Detected'])
    df['Last Detected'] = pd.to_datetime(df['Last Detected'])
    
    # Calculate the earliest and latest timestamps
    earliest_timestamp = df['First Detected'].min()
    latest_timestamp = df['Last Detected'].max()
    
    time_difference = latest_timestamp - earliest_timestamp

    return df, time_difference

In [ ]:
def sunburst_chart_output(df, path, maxdepth, chart_name, hover_name, hover_data, labels):
    fig = make_subplots(rows=1, cols=1)
    sunburst_chart = px.sunburst(
        df,
        path=path,
        values='Count',
        color='Count',  
        color_continuous_scale='Viridis', 
        hover_name=hover_name,  # Display Source app as hover text
        hover_data=hover_data,  
        labels=labels,  
        maxdepth=maxdepth,  
    )

    fig.add_trace(sunburst_chart.data[0])
    fig.update_layout(
        legend_title_text="Legend",
        legend_traceorder="normal",  # Change trace order in the legend
    )
    fig.update_traces(textinfo='label+percent entry', insidetextorientation='radial') 
    fig.update_layout(margin=dict(t=0, l=0, r=0, b=0))  

    chart_path = os.path.join(directory, chart_name)
    
    with open(chart_path, 'w') as f:
        f.write(fig.to_html())

    fig.show()

In [ ]:
def generate_sunburst_from_dataframe(df, columns, chart_name, path, hover_data, labels):
    global directory

    grouped = df.groupby(columns).size().reset_index(name='Count')

    if not grouped.empty:
        path1 = os.path.join(directory, f'{chart_name}_consolidated_output.csv')
        grouped.to_csv(path1, index=False)
    else:
        print("No data to generate Sunburst chart.")
        return
    
    path = path
    hover_name = path[-1]
    hover_data = hover_data
    labels = labels

    sunburst_chart_output(grouped, path, maxdepth, f'{chart_name}_sunburst_chart.html', hover_name, hover_data, labels)

In [ ]:
def generate_vens_to_vens_sunburst_chart(df, maxdepth=2, location=False):
    df_filtered = df[
        ((~df['Source Name'].isna()) | (~df['Source Hostname'].isna())) &
        ((~df['Destination Name'].isna()) | (~df['Destination Hostname'].isna()))
    ]
    
    columns = ['Source Application', 'Source Environment', 'Source Location', 'Destination Application', 'Destination Environment', 'Destination Location', 'Port', 'Protocol']
    
    if location:
        path = ['Destination Location', 'Destination Environment', 'Destination Application', 'Source Location', 'Source Environment', 'Source Application']
    else:
        path = ['Destination Environment', 'Destination Application', 'Source Environment', 'Source Application']

    hover_data = ['Count', 'Source Application', 'Source Environment', 'Destination Application', 'Destination Environment']
    labels = {'Source Application': 'Source App', 'Destination Application': 'Destination App'}

    generate_sunburst_from_dataframe(df_filtered, columns, 'vens-to-vens', path, hover_data, labels)

In [ ]:
def generate_iplists_to_vens_sunburst_chart(df, maxdepth=2, location=False):
    df_filtered = df[~df['Source IPList'].isna() & ~df['Destination Hostname'].isna()]
    columns = ['Source IPList', 'Destination Application', 'Destination Environment', 'Destination Location', 'Port', 'Protocol']

    if location:
        path = ['Destination Location', 'Destination Environment', 'Destination Application', 'Source IPList']
    else:
        path = ['Destination Environment', 'Destination Application', 'Source IPList']

    hover_data = ['Count', 'Source IPList', 'Destination Application', 'Destination Environment']
    labels = {'Source iplist': 'Source IPList', 'Destination Application': 'Destination App'}

    generate_sunburst_from_dataframe(df, columns, 'iplists-to-vens', path, hover_data, labels)

In [ ]:
def visualize_ports_usage(df, group, chart_name):
    grouped = df.groupby(group).size().reset_index(name='Count')
    stacked_chart_output(grouped, chart_name)
    table_view(df, group[0], group[-1])

In [ ]:
def table_view(df, Destination, Source):    
    # Create a table showing Destination app and its count
    Destination_app_counts = df[Destination].value_counts().reset_index()
    Destination_app_counts.columns = [Destination, 'Count']
    
    table_fig = ff.create_table(Destination_app_counts)
    table_fig.update_layout(
        title=f"{Destination} Unique Hits Counts",
        margin=dict(t=50, l=10, r=10, b=10)
    )
    
    Source_counts = df[Source].value_counts().reset_index()
    Source_counts.columns = [Source, 'Count']
    
    table_fig2 = ff.create_table(Source_counts)
    table_fig2.update_layout(
        title=f"{Source} Unique Hits Counts",
        margin=dict(t=50, l=10, r=10, b=10)
    )
    
    table_fig.show()
    table_fig2.show()

In [ ]:
def stacked_chart_output(grouped, chart_name):
    global directory
    
    # Create a new column to group ports within specified ranges
    def group_ports(port):
        if 0 <= port <= 1023:
            return '0-1023'
        elif 1024 <= port <= 49151:
            return '1024-49151'
        elif 49152 <= port <= 65535:
            return '49152-65535'
        else:
            return 'Other'

    grouped['Port Range'] = grouped['Port'].apply(group_ports)

    # Create three separate DataFrames for each port range
    low_ports = grouped[grouped['Port Range'] == '0-1023']
    medium_ports = grouped[grouped['Port Range'] == '1024-49151']
    high_ports = grouped[grouped['Port Range'] == '49152-65535']

    # Create three stacked bar charts using Plotly Express
    fig_low_ports = px.bar(low_ports, x='Destination Application', y='Count', color='Port', text='Count',
                           labels={'Destination Application': 'Destination Application', 'Count': 'Count'},
                           title=f'{chart_name} Count of Unique Hits (0-1023) per Destination Application (Stacked)')

    fig_medium_ports = px.bar(medium_ports, x='Destination Application', y='Count', color='Port', text='Count',
                              labels={'Destination Application': 'Destination Application', 'Count': 'Count'},
                              title=f'{chart_name} Count of Unique Hits (1024-49151) per Destination Application (Stacked)')

    fig_high_ports = px.bar(high_ports, x='Destination Application', y='Count', color='Port', text='Count',
                            labels={'Destination Application': 'Destination Application', 'Count': 'Count'},
                            title=f'{chart_name} Count of Unique Hits (49152-65535) per Destination Application (Stacked)')

    # Customize the appearance of all three charts
    for fig in [fig_low_ports, fig_medium_ports, fig_high_ports]:
        fig.update_traces(textposition='inside')

    chart_path = os.path.join(directory, chart_name)
        
    # Save the three charts as image files (PNG format)
    fig_low_ports.write_image(f"{chart_path}-fig_low_ports.png")
    fig_medium_ports.write_image(f"{chart_path}-fig_medium_ports.png")
    fig_high_ports.write_image(f"{chart_path}-fig_high_ports.png")

    # Show the charts
    fig_low_ports.show()
    fig_medium_ports.show()
    fig_high_ports.show()

In [ ]:
def generate_Source_and_Destination_hostname_without_applabel(df):
    global directory
    
    columns_to_replace = ['Source Application', 'Source Environment', 'Source Location', 'Destination Application', 'Destination Environment', 'Destination Location']
    df[columns_to_replace] = df[columns_to_replace].fillna('NO LABEL')
    
    # Filter and group the DataFrame based on 'Source app' and 'Destination app'
    Source_hostname_grouped = df[df['Source Application'] == 'NO LABEL'].groupby(['Source Hostname', 'Source IP', 'Source Application', 'Source Environment', 'Source Location']).size().reset_index(name='Count')
    Destination_hostname_grouped = df[df['Destination Application'] == 'NO LABEL'].groupby(['Destination Hostname', 'Destination IP', 'Destination Application', 'Destination Environment', 'Destination Location']).size().reset_index(name='Count')

    # Check if both Source_hostname_grouped and Destination_hostname_grouped are empty
    if Source_hostname_grouped.empty and Destination_hostname_grouped.empty:
        return "No concern", "No concern"
    
    # Save the grouped DataFrames to CSV files
    Source_csv_path = os.path.join(directory, 'source_hostname_without_applabel_output.csv')
    Destination_csv_path = os.path.join(directory, 'destination_hostname_without_applabel_output.csv')
    
    Source_hostname_grouped.to_csv(Source_csv_path, index=False)
    Destination_hostname_grouped.to_csv(Destination_csv_path, index=False)
    
    if Source_hostname_grouped.empty:
        return "No concern", Destination_hostname_grouped
    elif Destination_hostname_grouped.empty:
        return Source_hostname_grouped, "No concern"
    
    return Source_hostname_grouped, Destination_hostname_grouped

In [ ]:
def group_and_save_outputs(dataset, name):
    grouped = dataset.groupby(['Destination Application', 'Destination Environment', 'Destination Location'])

    # Iterate through each unique combination and create CSV files
    for group_keys, group_data in grouped:
        Destination_app, Destination_env, Destination_loc = group_keys

        Destination_app = Destination_app.replace('/','_')
        
        directory_name = f"{Destination_app} | {Destination_env} | {Destination_loc}"
        os.makedirs(directory_name, exist_ok=True)

        csv_filename = f"{name}_{Destination_app}_{Destination_env}_{Destination_loc}.csv"
        csv_path = os.path.join(directory_name, csv_filename)

        group_data.to_csv(csv_path, index=False)

In [ ]:
def process_csv_file(csv_file_path, output_prefix, columns):
    if os.path.exists(csv_file_path):
        df = read_csv_file(csv_file_path)
        visualize_ports_usage(df, columns, output_prefix)
        group_and_save_outputs(df, output_prefix)
    else:
        print(f"File not found: {csv_file_path}, skipping visualization.")

In [ ]:
def output_workloads(df, columns, filename):
    global directory
    
    # Group the DataFrame by the specified columns and count occurrences
    grouped = df.groupby(columns).size().reset_index(name='Count')
    grouped = grouped.sort_values(by='Count', ascending=False)
    
    csv_path = os.path.join(directory, filename)
    grouped.to_csv(csv_path, index=False)
    
    return grouped

<span style='background:Red;font-weight:bold;'> ### DO NOT EDIT TILL HERE ###</span>

In [ ]:
df = read_csv_file(csv_file_path)
df, time_difference = preprocess_data(df)

directory = "consolidated_outputs"
os.makedirs(directory, exist_ok=True)

In [ ]:
Source_hostname_without_applabel, Destination_hostname_without_applabel = generate_Source_and_Destination_hostname_without_applabel(df)

In [ ]:
if maxdepth < 2:
    maxdepth = 2
    
if maxdepth > 6:
    maxdepth = 6

 <span style='color:Blue;font-weight:bold;font-size: 20px'> Outputs:</span> 

In [ ]:
print(f"Dataset timeframe: {time_difference}")

<span style='font-size: 20px'> <span style='color:Blue;font-weight:bold'> INFO:</span> Port Hit Count (refer: "port_usage_count.csv")</span>

In [ ]:
output_workloads(df, ['Protocol', 'Port'], "port_usage_count.csv")
table_view(df, 'Protocol', 'Port')

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Outgoing Port Hit Count (refer: "outgoing_port_usage_count.csv")</span>

In [ ]:
print(output_workloads(df, ['Source Application', 'Source Environment', 'Source Location', 'Protocol', 'Port'], "outgoing_port_usage_count.csv"))

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Incoming Port Hit Count (refer: "incoming_port_usage_count.csv")</span>

In [ ]:
print(output_workloads(df, ['Destination Application', 'Destination Environment', 'Destination Location', 'Protocol', 'Port'], "incoming_port_usage_count.csv"))

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Source VENs/UMWLs Hit Count (refer: 'Source_hit_counts.csv')</span>

In [ ]:
print(output_workloads(df, ['Source Hostname', 'Source Application', 'Source Environment', 'Source Location'], 'Source_hit_counts.csv'))

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Destination VENs/UMWLs Hit Count (refer: 'Destination_hit_counts.csv')</span>

In [ ]:
print(output_workloads(df, ['Destination Hostname', 'Destination Application', 'Destination Environment', 'Destination Location'], 'Destination_hit_counts.csv'))

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Source VENs without APP label assigned (refer: 'Source_hostname_without_applabel_output.csv')</span>

In [ ]:
print(Source_hostname_without_applabel)

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Destination VENs without APP label assigned (refer: 'Destination_hostname_without_applabel_output.csv')</span>

In [ ]:
print(Destination_hostname_without_applabel)

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Sunburst Chart for VENs to VENs visualization <br>

This Sunburst chart displays a hierarchical representation of data. (refer: 'vens-to-vens_sunburst_chart.html')<br>

Layer 1: Destination environment<br>
Layer 2: Destination application within Destination environment<br>
Layer 3: Source environment outbound to the Destination Application<br>
Layer 4: Source application within Source environment<br></span>

In [ ]:
generate_vens_to_vens_sunburst_chart(df, maxdepth=maxdepth, location=location)

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> VENs to VENs Ports Usage by Destination App Centric</span>

In [ ]:
process_csv_file(f"{directory}/vens-to-vens_consolidated_output.csv", "VENs-to-VENs", ['Destination Application', 'Port', 'Protocol', 'Source Application'])

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> Sunburst Chart for IPLists to VENs visualization <br>

This Sunburst chart displays a hierarchical representation of data. (refer: 
'iplists-to-vens_sunburst_chart.html')<br>

Layer 1: Destination environment<br>
Layer 2: Destination application within Destination environment<br>
Layer 3: Source IPLists outbound to the Destination Application<br>
Layer 4: Source IPs within Source IPLists<br></span>

In [ ]:
generate_iplists_to_vens_sunburst_chart(df, maxdepth=maxdepth, location=location)

<span style='font-size: 20px'><span style='color:Blue;font-weight:bold;'> INFO:</span> IPLists to VENs Ports Usage by Destination App Centric</span>

In [ ]:
process_csv_file(f"{directory}/iplists-to-vens_consolidated_output.csv", "IPLists-to-VENs", ['Destination Application', 'Port', 'Protocol', 'Source IPList'])